#  Importing required libraries

In [1]:

import torch
import cv2
import time
import re
import numpy as np
import easyocr

# Initiating easyocr

In [2]:
##### DEFINING GLOBAL VARIABLE
EASY_OCR = easyocr.Reader(['en']) 
OCR_TH = 0.2

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


#  Function to run detection

In [3]:
def detectx (frame, model):
    frame = [frame]
    print(f"[INFO] Detecting. . . ")
    results = model(frame)
    labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

    return labels, cordinates

# To plot the BBox and results

In [4]:
def plot_boxes(results, frame,classes):

    """
    --> This function takes results, frame and classes
    --> results: contains labels and coordinates predicted by model on the given frame
    --> classes: contains the strting labels

    """
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    print(f"[INFO] Total {n} detections. . . ")
    print(f"[INFO] Looping through all detections. . . ")


    ### looping through the detections
    for i in range(n):
        row = cord[i]
        if row[4] >= 0.55: ### threshold value for detection. We are discarding everything below this value
            print(f"[INFO] Extracting BBox coordinates. . . ")
            x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape) ## BBOx coordniates
            text_d = classes[int(labels[i])]
            coords = [x1,y1,x2,y2]

            plate_num = recognize_plate_easyocr(img = frame, coords= coords, reader= EASY_OCR, region_threshold= OCR_TH)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) ## BBox
            cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255,0), -1) ## for text label background
            cv2.putText(frame, f"{plate_num}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255), 2)

    return frame

# Function to recognize license plate

In [5]:
# function to recognize license plate numbers using Tesseract OCR
def recognize_plate_easyocr(img, coords,reader,region_threshold):
    # separate coordinates from box
    xmin, ymin, xmax, ymax = coords
    # get the subimage that makes up the bounded region and take an additional 5 pixels on each side
    # nplate = img[int(ymin)-5:int(ymax)+5, int(xmin)-5:int(xmax)+5]
    nplate = img[int(ymin):int(ymax), int(xmin):int(xmax)] ### cropping the number plate from the whole image

    ocr_result = reader.readtext(nplate)

    text = filter_text(region=nplate, ocr_result=ocr_result, region_threshold= region_threshold)

    if len(text) ==1:
        text = text[0].upper()
    return text

### to filter out wrong detections 

def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    print(ocr_result)
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate


# Main function

In [6]:


def main(img_path=None, vid_path=None,vid_out = None):

    print(f"[INFO] Loading model... ")
    ## loading the custom trained model
    # model =  torch.hub.load('ultralytics/yolov5', 'custom', path='last.pt',force_reload=True) ## if you want to download the git repo and then run the detection
    model = torch.hub.load('yolov5', 'custom', source='local', path='yolov5/runs/train/yolov5s_results/weights/best.pt')

    classes = model.names ### class names in string format




    # For detection on image
    if img_path != None:
        print(f"[INFO] Working with image: {img_path}")
        img_out_name = f"./output/result_{img_path.split('/')[-1]}"

        frame = cv2.imread(img_path) ### reading the image
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        
        results = detectx(frame, model = model) ### DETECTION HAPPENING HERE    

        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)

        frame = plot_boxes(results, frame,classes = classes)
        

        cv2.namedWindow("img_only", cv2.WINDOW_NORMAL) ## creating a free windown to show the result

        while True:
            # frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)

            cv2.imshow("img_only", frame)

            if cv2.waitKey(5) & 0xFF == ord('q'):
                print(f"[INFO] Exiting. . . ")

                cv2.imwrite(f"{img_out_name}",frame) ## if you want to save he output result.

                break

    # For detection on video
    elif vid_path !=None:
        print(f"[INFO] Working with video: {vid_path}")

        ## reading the video
        cap = cv2.VideoCapture(vid_path)

        # Placeholder for the out variable
        out = None

        if vid_out:  # creating the video writer if video output path is given
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            codec = cv2.VideoWriter_fourcc(*'mp4v')  # or your desired codec
            out = cv2.VideoWriter(vid_out, codec, fps, (width, height))

        # assert cap.isOpened()
        frame_no = 1

        cv2.namedWindow("vid_out", cv2.WINDOW_NORMAL)
        while True:
            # start_time = time.time()
            ret, frame = cap.read()
            if ret  and frame_no %1 == 0:
                print(f"[INFO] Working with frame {frame_no} ")

                frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                results = detectx(frame, model = model)
                frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)


                frame = plot_boxes(results, frame,classes = classes)
                
                cv2.imshow("vid_out", frame)
                if vid_out:
                    print(f"[INFO] Saving output video. . . ")
                    out.write(frame)

                if cv2.waitKey(5) & 0xFF == ord('q'):
                    break
                frame_no += 1
        
        # Release the VideoWriter if it was created
        if out is not None:
            print(f"[INFO] Cleaning up. . . ")
            out.release()
        
        ## closing all windows
        cv2.destroyAllWindows()

In [7]:
main(img_path="img1.jpeg") #for image

[INFO] Loading model... 


requirements: YOLOv5 requirement "gitpython" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\ariji\Desktop\ANPR\yolov5\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  v7.0-72-g064365d Python-3.11.4 torch-2.1.0+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


[INFO] Working with image: img1.jpeg
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[19, 7], [177, 7], [177, 40], [19, 40]], 'UP16AJL567', 0.8161174996969421)]
[INFO] Exiting. . . 


In [7]:
main(vid_path='vid1.mp4') #for video

[INFO] Loading model... 


requirements: YOLOv5 requirement "gitpython" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\ariji\Desktop\ANPR\yolov5\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  v7.0-72-g064365d Python-3.11.4 torch-2.1.0+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


[INFO] Working with video: vid1.mp4
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[0, 3], [72, 3], [72, 21], [0, 21]], '4Ao8 BZM', 0.2433444130387395)]
[INFO] Extracting BBox coordinates. . . 
[([[4, 0], [124, 0], [124, 32], [4, 32]], 'L456 LhC', 0.4605901121061897)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[0, 3], [82, 3], [82, 23], [0, 23]], 'MAo8 BZM ', 0.09451382465690311)]
[INFO] Extracting BBox coordinates. . . 
[([[6, 0], [126, 0], [126, 32], [6, 32]], 'L456 LhC', 0.44600754602161874)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[1, 3], [87, 3], [87, 23], [1, 23]], 'MAoBBZM', 0.5024689903646907)]
[INFO] Extracting BBox coordinates. . . 
[([[4, 0], [124, 0], [124, 32], [4, 32]], 'L456 LH

[([[0, 0], [130, 0], [130, 32], [0, 32]], 'LS6LhC', 0.40753388758252296)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[0, 2], [92, 2], [92, 24], [0, 24]], 'MAoB BZM', 0.21848062718638886)]
[INFO] Extracting BBox coordinates. . . 
[([[1, 0], [135, 0], [135, 35], [1, 35]], 'LHSS LHC', 0.135286098371955)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[0, 2], [91, 2], [91, 24], [0, 24]], 'Mob BZM', 0.11291511524794338)]
[INFO] Extracting BBox coordinates. . . 
[([[5, 0], [139, 0], [139, 37], [5, 37]], 'L456 Lhc', 0.07463342870627662)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[1, 3], [91, 3], [91, 23], [1, 23]], 'HAoB BZM', 0.17989403895754374)]
[INFO] Extracting BBox coordinates. . . 
[([[3, 0], [

[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[0, 2], [82, 2], [82, 26], [0, 26]], 'MAoB BZ', 0.3157489349022913)]
[INFO] Extracting BBox coordinates. . . 
[([[5, 1], [146, 1], [146, 38], [5, 38]], 'LSsAC', 0.015522169802994932)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[4, 4], [84, 4], [84, 27], [4, 27]], 'MAo8 BZ', 0.20760958974593458)]
[INFO] Extracting BBox coordinates. . . 
[([[5, 1], [148, 1], [148, 38], [5, 38]], 'Ls64rC', 0.010413915875659163)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[0, 2], [79, 2], [79, 26], [0, 26]], 'Mo8 BZ', 0.35536348088391895)]
[INFO] Extracting BBox coordinates. . . 
[([[0, 0], [147, 0], [147, 39], [0, 39]], 'LS64C', 0.03001843538861907)]
[INFO] Detecting. . . 
[INFO] Total 2 det

[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Detecting. . . 
[INFO] Total 1 detections. . . 
[INFO] Loop

[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Extracting BBox coordinates. . . 
[]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through

[([[0, 0], [74, 0], [74, 22], [0, 22]], '2ARRIVE', 0.5443507014761118), ([[9, 25], [71, 25], [71, 38], [9, 38]], 'LJso AXF', 0.22378978566197177)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[8, 0], [59, 0], [59, 26], [8, 26]], 'Lpis', 0.6505225896835327)]
[INFO] Extracting BBox coordinates. . . 
[([[0, 0], [74, 0], [74, 24], [0, 24]], '2 ARRIVI', 0.679640836424968), ([[9, 25], [69, 25], [69, 39], [9, 39]], 'LJso AXF', 0.3439531950990709)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[6, 0], [59, 0], [59, 26], [6, 26]], 'Lpis', 0.4589846730232239)]
[INFO] Extracting BBox coordinates. . . 
[([[0, 0], [74, 0], [74, 24], [0, 24]], '2 ARRIVI', 0.597543752341882), ([[17, 27], [69, 27], [69, 39], [17, 39]], 'Jgo AXF', 0.25985959875293235)]
[INFO] Detecting. . . 
[INFO] Total 3 detections. . . 
[INF

[([[0, 3], [74, 3], [74, 23], [0, 23]], '2) ARRIVA', 0.42832860881683466)]
[INFO] Detecting. . . 
[INFO] Total 4 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[8, 0], [85, 0], [85, 30], [8, 30]], 'LeIs ', 0.2020652894688498)]
[INFO] Extracting BBox coordinates. . . 
[([[0, 0], [73, 0], [73, 24], [0, 24]], '2 ARRIVI', 0.9791836277245184)]
[INFO] Detecting. . . 
[INFO] Total 3 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[8, 0], [88, 0], [88, 30], [8, 30]], 'LPISVI', 0.13112222461802844)]
[INFO] Extracting BBox coordinates. . . 
[([[19, 5], [74, 5], [74, 25], [19, 25]], 'ARRIVA', 0.6626922039095465)]
[INFO] Detecting. . . 
[INFO] Total 3 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[12, 2], [91, 2], [91, 30], [12, 30]], 'LPIs VI', 0.29207331611940374)]
[INFO] Extracting BBox coordinates. . . 
[([[19, 3], [76, 3]

[([[7, 5], [63, 5], [63, 21], [7, 21]], 'LJeol', 0.2766735830913965)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[12, 4], [126, 4], [126, 36], [12, 36]], 'LPIS DV', 0.7434226547174635)]
[INFO] Extracting BBox coordinates. . . 
[([[9, 5], [49, 5], [49, 21], [9, 21]], 'LJbq', 0.2938011884689331)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[12, 4], [123, 4], [123, 36], [12, 36]], 'LPIS NDI', 0.18593193657942417)]
[INFO] Extracting BBox coordinates. . . 
[([[19, 7], [60, 7], [60, 22], [19, 22]], 'Jeoa', 0.42025551199913025)]
[INFO] Detecting. . . 
[INFO] Total 2 detections. . . 
[INFO] Looping through all detections. . . 
[INFO] Extracting BBox coordinates. . . 
[([[12, 4], [118, 4], [118, 36], [12, 36]], 'LPIS D', 0.4940470989454642)]
[INFO] Extracting BBox coordinates. . . 
[([[19, 7], [47, 